# 🤖 在Jupyter Notebook中玩转Qwen3-VL视觉大模型：从部署到对话全指南

随着大模型的应用越来越广泛，相信大家已经不再满足于只能与助手聊天的单一功能，对大模型处理图像的需求也日渐变多。本教程将手把手带你，在熟悉的Jupyter Notebook环境中，通过llama.cpp轻松部署强大的Qwen3-VL多模态模型。告别复杂的命令行和云服务依赖，只需一步步跟随，即可在本地构建一个支持图像理解与智能对话的私人助手。
核心优势：
- **完全本地化**：模型在本地运行，无需网络，隐私无忧。

- **多模态能力**：不仅会聊天，更能理解你发送的图片内容。

- **Notebook友好**：所有操作均在Jupyter单元格内完成，交互直观。

- **免费开源**：依托llama.cpp和ModelScope开源社区。

## 🎯 极速预览：成功部署“三步曲”
如果你喜欢直入主题，以下是已验证的成功路径摘要：（---在终端中执行命令---）
### 第1步：从ModelScope下载Qwen3模型文件
```python
pip install modelscope --upgrade
modelscope download --model 'Qwen/Qwen3-VL-2B-Instruct-GGUF' Qwen3VL-2B-Instruct-Q4_K_M.gguf mmproj-Qwen3VL-2B-Instruct-F16.gguf (下载到默认cache地址)
```

### 第2步：获取并编译模型引擎 (llama.cpp)
```python
git clone https://github.com/ggml-org/llama.cpp
cd llama.cpp
cmake -B build
cmake --build build --config Release -j $(nproc 2>/dev/null || sysctl -n hw.ncpu 2>/dev/null || echo 4)
```

### 第3步：启动本地服务器（替换实际路径）
```
  ./build/bin/llama-server \
  -m /your/path/to/Qwen3VL-2B-Instruct-Q4_K_M.gguf \
  --mmproj /your/path/to/mmproj-Qwen3VL-2B-Instruct-F16.gguf \
  -c 4096 \
  --host 0.0.0.0 \
  --port 8080
  ```

### 第4步：启动模型交互界面（可选）
```
./build/bin/llama-cli \
-m /your/path/to/Qwen3VL-2B-Instruct-Q4_K_M.gguf \
--mmproj /your/path/to/mmproj-Qwen3VL-2B-Instruct-F16.gguf \
--jinja \
--color auto
```

### 注意----第一步模型下载和第三步的启动server可以用一步来搞定，即：
但是要注意加上`--mmproj`参数来指定投影文件，启用多模态
```
export MODEL_ENDPOINT=https://www.modelscope.cn/
./build/bin/llama-server -hf Qwen/Qwen3-VL-2B-Instruct-GGUF --jinja -ngl 99 -fa auto -sm row --temp 0.6 --top-k 20 --top-p 0.95 --min-p 0 -c 40960 -n 32768 --no-context-shift
```
类似的，第一步和第四步也可以合并为一步：
```
export MODEL_ENDPOINT=https://www.modelscope.cn/
./build/bin/llama-cli -hf Qwen/Qwen3-VL-2B-Instruct-GGUF --jinja --color auto -ngl 99 -fa auto -sm row --temp 0.6 --top-k 20 --top-p 0.95 --min-p 0 -c 40960 -n 32768 --no-context-shift
```
（这样通过设置`MODEL_ENDPOINT`环境变量，可以让程序从ModelScope下载模型，下载完成之后直接启动）\
下面，我们将详细展开每一步在Notebook中的使用。

## 📦 第一章：战前准备（环境与模型）

### 1.1 认识llama.cpp
开始之前，我们先简要介绍一下 llama.cpp，说明你可以从中获得什么，以及为什么我们要强调“使用” llama.cpp。本质上，llama.cpp 是一个独立的生态系统，其设计理念注重轻量化、最小化外部依赖、支持多平台，并提供广泛而灵活的硬件兼容性：

- 纯 C/C++ 实现，无外部依赖

- 支持广泛的硬件：

  - x86_64 CPU 支持 AVX、AVX2 与 AVX512

  - 通过 Metal 与 Accelerate 支持 Apple Silicon（CPU 与 GPU）

  - 支持 NVIDIA GPU（通过 CUDA）、AMD GPU（通过 hipBLAS）、Intel GPU（通过 SYCL）、昇腾 NPU（通过 CANN）以及摩尔线程 GPU（通过 MUSA）

  - 提供 GPU 的 Vulkan 后端

- 提供多种量化方案，以提升推理速度并降低内存占用

- 支持 CPU+GPU 混合推理，可加速运行超过显存容量的模型

它类似于使用 Python 框架 `torch`+`transformers` 或 `torch`+`vllm`，但以 C++ 实现。然而，两者之间存在重要区别：

- Python 是一种解释型语言：代码由解释器逐行执行，你可以通过解释器或交互式终端直接运行代码片段或脚本。Python 对初学者友好，即使了解不深，也较容易修改源码。

- C++ 是一种编译型语言：源代码需先经编译转换为机器码与可执行文件，语言层面的开销极小。llama.cpp 同样提供了示例程序的源代码，展示如何使用该库。但若你不熟悉 C++ 或 C，修改源码将较为困难。

真正“使用” llama.cpp 意味着将其作为库集成到自己的程序中，类似 Ollama、LM Studio、GPT4ALL、llamafile 等项目的实现方式。但这并非本指南的目标或所能覆盖的内容。这里我们将主要介绍如何使用 `llama-server` 和 `llama-cli` 示例程序，帮助你了解 llama.cpp 对 Qwen2.5 模型的支持，以及 llama.cpp 生态系统的基本运作方式。



### 1.2 Qwen3-VL介绍
#### [认识Qwen3-VL系列模型](https://modelscope.cn/models/Qwen/Qwen3-VL-2B-Instruct) — 迄今为止 Qwen 系列中功能最强大的视觉语言模型。
这一代产品在各个方面都进行了全面升级：更优秀的文本理解和生成能力、更深入的视觉感知和推理能力、扩展的上下文长度、增强的空间和视频动态理解能力，以及更强的代理交互能力。\
提供从边缘到云端可扩展的 Dense 和 MoE 架构，并提供 Instruct 和增强推理的 Thinking 版本，以实现灵活、按需部署。\
主要增强功能：
- 视觉代理：操作 PC/移动 GUI — 识别元素、理解功能、调用工具、完成任务。
- 视觉编码增强：从图像/视频生成 Draw.io/HTML/CSS/JS。
- 高级空间感知：判断物体位置、视角和遮挡；提供更强的 2D 基础，并支持 3D 基础，用于空间推理和具身 AI。
- 长上下文和视频理解：原生 256K 上下文，可扩展至 1M；处理书籍和数小时的视频，具有完整的回忆和秒级索引。
- 增强的多模态推理：在 STEM/数学方面表现出色 — 因果分析和基于逻辑、证据的答案。
- 升级的视觉识别：更广泛、更高品质的预训练能够“识别一切”——名人、动漫、产品、地标、动植物等。
- 扩展的 OCR：支持 32 种语言（从 19 种增加）；在低光、模糊和倾斜情况下表现稳健；更好地处理罕见/古代字符和术语；改进了长文档结构解析。
- 与纯 LLM 相当的文本理解：无缝的文本-视觉融合，实现无损、统一的理解

### 1.3 GGUF介绍
GGUF是一种文件格式，用于存储运行模型所需的信息，包括但不限于模型权重、模型超参数、默认生成配置和tokenzier。\
在[Qwen3-VL-2B-Instruct-GGUF](https://modelscope.cn/models/Qwen/Qwen3-VL-2B-Instruct-GGUF/summary)模型仓库中，仓库提供了 Qwen3-VL-2B-Instruct 的 GGUF 格式权重，分为两个组件：
- 语言模型 (LLM)：FP16, Q8_0, Q4_K_M
- 视觉编码器 (mmproj)：FP16, Q8_0
这些文件与 llama.cpp、Ollama 和其他基于 GGUF 的工具兼容，支持在 CPU、NVIDIA GPU (CUDA)、Apple Silicon (Metal)、Intel GPUs (SYCL) 等上进行推理。您可以根据您的硬件和性能需求混合使用语言和视觉组件的精度级别，甚至可以从 FP16 权重开始进行自定义量化。

### 1.4 获取“AI大脑”：模型文件
Qwen3-VL需要两个核心文件，分别负责“语言”和“视觉”：
- 主模型文件 (*.gguf)：语言理解与生成的核心。
- 视觉投影文件 (mmproj-*.gguf)：将图像信息转换为模型可理解的“语言”。
- 模型文件：`GGUF格式`极大简化了大语言模型文件的管理，可通过单模型文件完成推理。而且借助llama.cpp提供的丰富量化能力，一个模型repo下的不同GGUF文件，通常对应的是不同量化精度与量化方法。本教程默认选用的是Q4_K_M版本，在推理精度以及推理速度，资源消耗之间做一个较好的均衡。如果有特殊的需求，也可以选择更高的精度--例如FP16版本。
- tips：本文用到的是`Qwen/Qwen3-VL-2B-Instruct-GGUF`模型，[点击即可跳转查看完整的模型文件](https://modelscope.cn/models/Qwen/Qwen3-VL-2B-Instruct-GGUF/files)。
  
我们可以从[ModelScope社区下载模型文件](https://modelscope.cn/docs/models/download)(以下两种方式任选一种即可)：

In [ ]:
# 简洁版 ---- 一行命令下载所需文件（使用命令行工具下载）
!modelscope download --model 'Qwen/Qwen3-VL-2B-Instruct-GGUF' Qwen3VL-2B-Instruct-Q4_K_M.gguf mmproj-Qwen3VL-2B-Instruct-F16.gguf

In [ ]:
# 安装下载工具 使用 ModelScope SDK 下载
!pip install modelscope --upgrade

import os
from modelscope import model_file_download

# 指定模型 (以2B参数、平衡量化版为例)
model_id = 'Qwen/Qwen3-VL-2B-Instruct-GGUF'
gguf_file = 'Qwen3VL-2B-Instruct-Q4_K_M.gguf'
mmproj_file = 'mmproj-Qwen3VL-2B-Instruct-F16.gguf'

print("⏳ 开始下载模型文件 (约1-2GB)，请稍候...")
model_path = model_file_download(model_id, gguf_file)
mmproj_path = model_file_download(model_id, mmproj_file)

print(f"\n🎉 下载完成！")
print(f"   语言模型: {model_path}")
print(f"   视觉模型: {mmproj_path}")

- 注意：这里输出的两个文件路径后面会用到，后续也可以尝试已经获取的`model_path`和`mmproj_path`变量。

## 🔧 第二章：编译llama.cpp引擎
llama.cpp是一个高效运行大模型的C++工具库，我们需要将它编译成可执行文件。

### 2.1 获取源码
在Notebook新单元格中执行：

In [ ]:
!git clone https://github.com/ggml-org/llama.cpp.git
%cd llama.cpp
print("源代码克隆完成！")

### 2.2 一键编译

In [ ]:
# 随时使用pwd命令查看你的当前工作目录
!pwd

In [ ]:
# 创建并进入构建目录
!cmake -B build
!cmake --build build --config Release

## 🚀 第三章：启动你的AI助手
这是最关键的一步，我们将启动模型服务。

### 3.1 理解启动“密语”
启动多模态服务需要两个关键指令，缺一不可：

- 主模型路径 (-m)：指向下载的.gguf主模型文件;
- 视觉投影路径 (--mmproj)：指向下载的mmproj文件。
- tips:因为我们这里使用的是`-m model.gguf`的方式来使用llama.cpp的，所以需要使用`--mmproj file.gguf`额外指定**视觉投影文件**
- 具体的&额外的使用方法可以参考[llama.cpp官方文档](https://github.com/ggml-org/llama.cpp/blob/master/docs/multimodal.md)

### 3.2 在Notebook中优雅地启动
llama-server 是一个简单的 HTTP 服务器，包含一组 LLM REST API 和一个简单的 Web 前端，用于通过 llama.cpp 与大型语言模型交互。\
默认情况下，服务器将在`http://localhost:8080`监听。所以你也可以打开网页，通过llama.cpp与qwen3-vl模型进行交互。\
为了更好管理，我们使用一个封装函数启动llama-server：

In [ ]:
import subprocess, os, time, requests

def start_ai_server():
    """一键启动Qwen3-VL多模态AI服务器"""
    print("="*50)
    print("🚀 正在启动你的Qwen3-VL AI助手服务器")
    print("="*50)
    
    # 1. 如果遇到路径错误，请修改为你的实际文件路径！查看在1.4中的输出结果，或者尝试已经获取的`model_path`和`mmproj_path`变量。
    # server可以使用相对路径，例如`build/bin/llama-server`。
    # 模型文件我们使用已经获取的`model_path`和`mmproj_path`变量。
    # 后续的路径同理。
    paths = {
        'server': './build/bin/llama-server',
        'model': model_path,
        'mmproj': mmproj_path
    }
    
    # 2. 组装启动命令
    cmd = [
        str(paths['server']), '-m', str(paths['model']),
        '--mmproj', str(paths['mmproj']),  # 多模态的关键！
        '-c', '4096',  # 上下文长度，可根据需要调小以节省内存
        '--host', '0.0.0.0', '--port', '8080'
    ]
    
    # 3. 启动！
    print("\n⏳ 启动服务器进程中...（首次加载模型需要1-3分钟，请稍候）")
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    # 等待并检查
    time.sleep(5)  # 短暂等待进程启动
    print("\n⏳ 正在等待服务器就绪...（预计需要1-3分钟）")
    max_wait = 180  # 最长等待3分钟
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        try:
            resp = requests.get("http://localhost:8080/health", timeout=5)
            if resp.status_code == 200:
                print(f"\n🎊 服务器启动成功！")
                print(f"   📍 本地访问: http://localhost:8080")
                print(f"   💬 API已就绪: http://localhost:8080/v1/chat/completions")
                return process
        except requests.exceptions.RequestException:
            # 服务器尚未就绪，继续轮询
            pass
        time.sleep(10)  # 等待10秒后重试
    
    print("⏳ 服务器启动超时或仍在加载。请检查子进程输出。")
    print("   完成后可手动在浏览器访问 http://localhost:8080 查看")
    return process

# 执行启动
my_ai_server = start_ai_server()

### 3.3 同样的，我们可以将模型下载和启动服务合并为一步来执行
需要注意的是，这种方式我们还是需要指定一下投影文件，使用 `--mmproj` 参数，如果不指定投影文件的话，就相当于[禁用了多模态](https://github.com/ggml-org/llama.cpp/blob/master/docs/multimodal.md)，只能对话聊天，不能发送图片。\
并且，使用`-hf`的方式，下载Qwen/Qwen3-VL-2B-Instruct-GGUF模型，只会下载Qwen3VL-2B-Instruct-Q4_K_M.gguf版本的模型文件，所以还需要手动下载投影文件（可以参考1.4中的下载方式）。

In [ ]:
import subprocess, os, time, requests
from pathlib import Path

def start_ai_server():
    """一键启动Qwen3-VL多模态AI服务器"""
    print("="*50)
    print("🚀 正在启动你的Qwen3-VL AI助手服务器")
    print("="*50)

    # 1. 设置环境变量，从ModelScope下载模型
    env = os.environ.copy()
    env['MODEL_ENDPOINT'] = 'https://www.modelscope.cn/'
    
    # 1. 如果遇到路径错误，请修改为你的实际server和mmproj文件路径！
    # 模型文件我们使用已经获取的`mmproj_path`变量。
    path_server = './build/bin/llama-server'
    path_mmproj = mmproj_path
    
    # 3. 组装启动命令
    cmd = [
        path_server,
        '-hf', 'Qwen/Qwen3-VL-2B-Instruct-GGUF',
        '--mmproj', path_mmproj,
        '--jinja',
        '-ngl', '99',
        '-fa', 'auto',
        '-sm', 'row',
        '--temp', '0.6',
        '--top-k', '20',
        '--top-p', '0.95',
        '--min-p', '0',
        '-c', '4096',
        '-n', '32768',
        '--no-context-shift',
    ]

    
    # 4. 启动！
    print("\n⏳ 启动服务器进程中...（首次加载模型需要1-3分钟，请稍候）")
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, env=env, stderr=subprocess.STDOUT, text=True)
    
    # 等待并检查
    time.sleep(5)  # 短暂等待进程启动
    print("\n⏳ 正在等待服务器就绪...（预计需要1-3分钟）")
    max_wait = 180  # 最长等待3分钟
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        try:
            resp = requests.get("http://localhost:8080/health", timeout=5)
            if resp.status_code == 200:
                print(f"\n🎊 服务器启动成功！")
                print(f"   📍 本地访问: http://localhost:8080")
                print(f"   💬 API已就绪: http://localhost:8080/v1/chat/completions")
                return process
        except requests.exceptions.RequestException:
            # 服务器尚未就绪，继续轮询
            pass
        time.sleep(10)  # 等待10秒后重试
    
    print("⏳ 服务器启动超时或仍在加载。请检查子进程输出。")
    print("   完成后可手动在浏览器访问 http://localhost:8080 查看")
    return process

# 执行启动
my_ai_server = start_ai_server()

当然，你也可以通过我们开头提到的，`极速预览`中的第三步，直接在**终端Terminal中**使用命令行的方式来启动服务器，效果是一样的。
```python
  ./build/bin/llama-server \
  -m /your/path/to/Qwen3VL-2B-Instruct-Q4_K_M.gguf \
  --mmproj /your/path/to/modelscope/mmproj-Qwen3VL-2B-Instruct-F16.gguf \
  -c 4096 \
  --host 0.0.0.0 \
  --port 8080
  ```

## 🧪 第四站：对话测试 & 炫酷应用
服务器跑起来后，让我们试试效果。

### 4.1 基础测试：让AI描述你的图片

In [ ]:
import requests, base64, json

def ask_ai_about_image(image_path, question="描述这张图片"):
    """发送图片和问题给AI助手"""
    # 将图片转为Base64
    with open(image_path, "rb") as f:
        img_b64 = base64.b64encode(f.read()).decode('utf-8')
    
    # 构建请求
    url = "http://localhost:8080/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "messages": [{
            "role": "user",
            "content": [
                {"type": "text", "text": question},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
            ]
        }],
        "max_tokens": 300
    }
    
    print(f"📤 询问AI: '{question}'")
    resp = requests.post(url, json=data, headers=headers, timeout=60)
    
    if resp.status_code == 200:
        reply = resp.json()['choices'][0]['message']['content']
        print(f"\n🤖 AI助手回复:\n{'-'*40}\n{reply}\n{'-'*40}")
        return reply
    else:
        print(f"请求出错: {resp.status_code}")
        return None

# 这里我们下载一张边牧的图片，可以不下载(注释掉)替换为你的图片路径，也可以修改问题
!wget -q --show-progress https://modelscope.oss-cn-beijing.aliyuncs.com/Dog.png -O dog.png
ask_ai_about_image("dog.png", "图片里有什么？")

### 4.2 使用OpenAI格式进行对话请求
其中，api_key和model字段此处没有作用，但是要填上

#### 初始化客户端

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:8080/v1',
    api_key='not-needed'
)

#### 纯文本聊天

In [ ]:
# 简单提问
response = client.chat.completions.create(
    model='qwen3-vl',
    messages=[{
        'role': 'user',
        'content': '你好，请介绍一下你自己'
    }]
)

print(f"Q: 你好，请介绍一下你自己")
print(f"A: {response.choices[0].message.content}")

#### 多模态--带图片的聊天

In [ ]:
import base64

# 读取图片并转换为Base64
image_path = "dog.png"  # 替换为你的图片路径，或使用之前下载的 dog.png

with open(image_path, "rb") as f:
    img_b64 = base64.b64encode(f.read()).decode('utf-8')

# 发送流式请求
response = client.chat.completions.create(
    model='qwen3-vl',
    messages=[{
        'role': 'user',
        'content': [
            {'type': 'text', 'text': '详细描述这张图片的内容'},
            {
                'type': 'image_url',
                'image_url': {
                    'url': f'data:image/jpeg;base64,{img_b64}'
                }
            }
        ]
    }],
    stream=True,  # 启用流式响应
    max_tokens=500,
    temperature=0.7
)

print("正在分析图片，流式回复开始：")
print("-" * 50)

full_response = ""
for chunk in response:
    if chunk.choices and chunk.choices[0].delta.content:
        content = chunk.choices[0].delta.content
        print(content, end='', flush=True)  # 逐字显示
        full_response += content

### 4.3 使用curl进行请求

In [ ]:
%%bash
curl -X POST http://localhost:8080/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
  "model": "qwen3-vl",
  "messages": [
    {
      "role": "user",
      "content": "你好，请介绍一下你自己"
    }
  ],
  "max_tokens": 300,
  "temperature": 0.6
}'

In [ ]:
%%bash
# 这里可以选择将图片编码，或者选择在下面的请求中，将图片地址传入'url'参数，我们下载一张奥黛丽·赫本的图片为例
wget -q --show-progress https://modelscope.oss-cn-beijing.aliyuncs.com/demo/images/audrey_hepburn.jpg -O audrey_hepburn.jpg
IMAGE_B64=$(base64 -i audrey_hepburn.jpg | tr -d '\n' | sed 's/"/\\"/g; s/\\/\\\\/g')

curl -X POST http://localhost:8080/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
  "model": "qwen3-vl",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "图片里有什么？"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "data:image/jpg;base64,'"${IMAGE_B64}"'" 
          }
        }
      ]
    }
  ],
  "max_tokens": 300,
  "temperature": 0.6
}'


#### 健康检查：
随时运行以下代码，检查助手是否“健康在线”：

In [ ]:
import requests
try:
    resp = requests.get("http://localhost:8080/health", timeout=3)
    print(f"✅ AI助手状态健康 (HTTP {resp.status_code})")
except:
    print("❌ AI助手服务未响应，请检查是否已启动。")

## 📚 总结与展望
恭喜你！🎉 至此，你已经成功在本地部署了一个功能强大的多模态AI助手。我们来回顾一下核心要点：

- **核心认知**：部署Qwen3-VL需要 “两个文件”（主模型+视觉投影)。

- **核心步骤**：准备模型 -> 编译引擎 -> 启动服务 -> 对话测试。

- **关键技巧**：在Notebook中使用subprocess管理进程，使用绝对路径，并对首次加载保持耐心。

资源导航：

- **[llama.cpp官方GitHub](https://github.com/ggml-org/llama.cpp)**：关注更新，获取文档。

- **[ModelScope模型社区](https://modelscope.cn/models)**：海量开源模型免费下载。

- **[Qwen官方文档](https://qwen.readthedocs.io/zh-cn/latest/run_locally/llama.cpp.html)**：深入了解模型特性。

希望本教程能帮助你顺利开启本地AI探索之旅。祝你玩得开心！✨

